
### <font color="Red" size="6"> Juan Esteban Moreno Agudelo</font>
#***Solución del problema 1***

### <font color="blue" size="7"> 
            Taller de programación entera 
</font>

### 1. Considere el problema de programación entera

> #### $ min\,\, x_{1} + 2x_{2}$

### Sujeto a:
> ##### $x_{1} + x_{2} \geq 4$
> ##### $\frac{1}{2}x_{1} + \frac{5}{2}x_{2} \geq \frac{5}{2}$

> ##### $x_{1},\, x_{2}\,\,$ *enteras* 

### <font color="red"> 1.1. Resolvemos la relajación lineal </font>

#### La relajación lineal es simplemente no restringir a enteras las variables del problema.

In [1]:
!pip install pyomo 
from matplotlib import pyplot as plt
import pyomo.environ as pyo
from pyomo.opt import SolverFactory, SolverStatus, TerminationCondition
import pandas as pd
import numpy as np
import copy

     |████████████████████████████████| 9.4MB 5.2MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 256kB 47.8MB/s 
     |████████████████████████████████| 163kB 47.7MB/s 


In [2]:
# instal cbc
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 144793 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.9.9+repack1-1_amd64.deb ...
Unpacking coinor-libcbc3 (2.9.9+repack1-1) ...
Selecting p

### Problema original

In [9]:
# Original problem
model = pyo.ConcreteModel()
model.x1 = pyo.Var(domain=pyo.PositiveReals)
model.x2 = pyo.Var(domain=pyo.PositiveReals)
model.x3 = pyo.Var(domain=pyo.PositiveReals)
model.x4 = pyo.Var(domain=pyo.PositiveReals)
model.x5 = pyo.Var(domain=pyo.PositiveReals)

model.objective = pyo.Objective(expr= -10*model.x1 - 6*model.x2 + 4, sense=pyo.minimize)
model.con1 = pyo.Constraint(expr=(1/11)*model.x1 + (2/11)*model.x2 + (1/11)*model.x3 == 6)
model.con2 = pyo.Constraint(expr=(2/11)*model.x1 - (1/11)*model.x2 + (1/11)*model.x4 == 4)
model.con3 = pyo.Constraint(expr=(5/11)*model.x1 + (3/11)*model.x2 + (1/11)*model.x5 == 15)

opt = pyo.SolverFactory("cbc", executable='/usr/bin/cbc')
results = opt.solve(model)  
model.display()


Model unknown

  Variables:
    x1 : Size=1, Index=None
        Key  : Lower : Value     : Upper : Fixed : Stale : Domain
        None :     0 : 18.857143 :  None : False : False : PositiveReals
    x2 : Size=1, Index=None
        Key  : Lower : Value     : Upper : Fixed : Stale : Domain
        None :     0 : 23.571429 :  None : False : False : PositiveReals
    x3 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : PositiveReals
    x4 : Size=1, Index=None
        Key  : Lower : Value     : Upper : Fixed : Stale : Domain
        None :     0 : 29.857143 :  None : False : False : PositiveReals
    x5 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : PositiveReals

  Objectives:
    objective : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : -326.000004

  Constraints:
    con1

#### La solución del problema restringido a los enteros es el siguiente.

> ### $(\frac{15}{4}, \frac{1}{4}),\,\,\,\, z=4.25$

#### Se puede ver en la siguiente gráfica.

In [ ]:
# Modified problem
model = pyo.ConcreteModel()
model.x1 = pyo.Var(domain=pyo.PositiveReals)
model.x2 = pyo.Var(domain=pyo.PositiveReals)

# Not basic variables
# model.S1 = pyo.Var(domain=pyo.PositiveReals)
# model.S2 = pyo.Var(domain=pyo.PositiveReals)

model.objective = pyo.Objective(expr=model.x1 + 2*model.x2, sense=pyo.minimize)
model.con1 = pyo.Constraint(expr=model.x1 + model.x2  >= 4)
model.con2 = pyo.Constraint(expr=(1/2)*model.x1 + (5/2)*model.x2 >= (5/2))

# Gomory-Chvátal cuts
model.con3 = pyo.Constraint(expr=model.x1 + 3*model.x2 >= 5)
#model.con3 = pyo.Constraint(expr=model.x2 <= 0)

opt = pyo.SolverFactory("cbc", executable='/usr/bin/cbc')
results = opt.solve(model)  

In [ ]:
model.display()

Model unknown

  Variables:
    x1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   3.5 :  None : False : False : PositiveReals
    x2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.5 :  None : False : False : PositiveReals

  Objectives:
    objective : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :   4.5

  Constraints:
    con1 : Size=1
        Key  : Lower : Body : Upper
        None :   4.0 :  4.0 :  None
    con2 : Size=1
        Key  : Lower : Body : Upper
        None :   2.5 :  3.0 :  None
    con3 : Size=1
        Key  : Lower : Body : Upper
        None :   5.0 :  5.0 :  None


In [ ]:
model = pyo.ConcreteModel()
model.x1 = pyo.Var(domain=pyo.PositiveReals)
model.x2 = pyo.Var(domain=pyo.PositiveReals)

model.x3 = pyo.Var(domain=pyo.PositiveReals)
model.x4 = pyo.Var(domain=pyo.PositiveReals)

model.objective = pyo.Objective(expr=1.00*model.x1 + 0.64*model.x2, sense=pyo.maximize)
model.con1 = pyo.Constraint(expr=50*model.x1 + 31*model.x2 + model.x3 == 250)
model.con2 = pyo.Constraint(expr=3*model.x1 - 2*model.x2 - model.x4 == -4)
# model.con3 = pyo.Constraint(expr=(3)*model.x3 + 50*model.x4 >= 178)

opt = pyo.SolverFactory("cbc", executable='/usr/bin/cbc')
results = opt.solve(model) 
model.display()

Model unknown

  Variables:
    x1 : Size=1, Index=None
        Key  : Lower : Value     : Upper : Fixed : Stale : Domain
        None :     0 : 1.9481865 :  None : False : False : PositiveReals
    x2 : Size=1, Index=None
        Key  : Lower : Value     : Upper : Fixed : Stale : Domain
        None :     0 : 4.9222798 :  None : False : False : PositiveReals
    x3 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : PositiveReals
    x4 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : PositiveReals

  Objectives:
    objective : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 5.098445572

  Constraints:
    con1 : Size=1
        Key  : Lower : Body        : Upper
        None : 250.0 : 249.9999988 : 250.0
    con2 : Size=1
        Key  : Lower : Body       : Upper
        Non

In [ ]:
# model.objective.expr()

In [ ]:
# model.x2.display()

In [ ]:
model = pyo.ConcreteModel()
# model.x1 = pyo.Var(bounds=(0,1))
# model.x2 = pyo.Var(bounds=(0,1))
# model.x3 = pyo.Var(bounds=(0,1))
# model.x4 = pyo.Var(bounds=(0,1))

model.x1 = pyo.Var(domain=pyo.Binary)
model.x2 = pyo.Var(domain=pyo.Binary)
model.x3 = pyo.Var(domain=pyo.Binary)
model.x4 = pyo.Var(domain=pyo.Binary)

model.objective = pyo.Objective(expr=17*model.x1 + 10*model.x2 + 25*model.x3 + 17*model.x4, sense=pyo.maximize)
model.con1 = pyo.Constraint(expr=5*model.x1 + 3*model.x2 +8*model.x3 + 7*model.x4 <= 12)

####### Nodo 3
# model.con3 = pyo.Constraint(expr=model.x3 == 1)
# model.con4 = pyo.Constraint(expr=model.x2 == 1)
# model.con5 = pyo.Constraint(expr=model.x1 == 1)
# model.con6 = pyo.Constraint(expr=model.x4 == 1)

####### Nodo 4
# model.con6 = pyo.Constraint(expr=model.x4 == 0)
# model.con7 = pyo.Constraint(expr=model.x4 == 1)

###### Nodo 2
# model.con2 = pyo.Constraint(expr=model.x3 == 0)
# model.con3 = pyo.Constraint(expr=model.x4 == 1)

opt = pyo.SolverFactory("cbc", executable='/usr/bin/cbc')
results = opt.solve(model)  

In [ ]:
model.display()

Model unknown

  Variables:
    x1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :     1 : False : False : Binary
    x2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   1.0 :     1 : False : False : Binary
    x3 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   1.0 :     1 : False : False : Binary
    x4 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :     1 : False : False : Binary

  Objectives:
    objective : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :  35.0

  Constraints:
    con1 : Size=1
        Key  : Lower : Body : Upper
        None :  None : 11.0 :  12.0
    con3 : Size=1
        Key  : Lower : Body : Upper
        None :   1.0 :  1.0 :   1.0
